In [1]:
using GLPK
using JuMP

In [2]:
m = Model()
set_optimizer(m,GLPK.Optimizer);

In [3]:
mutable struct Instance
    n :: Int64
    D
    l
    function Instance(s)
        D = zeros(1,1)
        nNodes = 0
        l = 0
        open(s) do f
            line = 1
            lineContent = readline(f)
            lineSplited = split(lineContent, " ")
            nNodes = parse(Int64, lineSplited[1])
            l = parse(Int64, lineSplited[2])
            D = zeros(nNodes,nNodes)
            while ! eof(f)
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                for col = 1:nNodes
                    D[line,col] = parse(Float64, lineSplited[col])
                end
                
                line += 1
            end
        end
        new(nNodes,D,l)
    end
end

In [4]:
I = Instance("instances/test.txt")

Instance(5, [0.0 2.0 … 3.0 1.0; 2.0 0.0 … 1.0 1.0; … ; 3.0 1.0 … 0.0 1.0; 1.0 1.0 … 1.0 0.0], 1)

In [5]:
# @variable(m, u[1:I.n]) # colocar real positivo # restrição remoção de subciclos
@variable(m, x[1:I.n,1:I.n], Bin)
@variable(m, y[1:I.n,1:I.n], Int)

5×5 Matrix{VariableRef}:
 y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
 y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]
 y[3,1]  y[3,2]  y[3,3]  y[3,4]  y[3,5]
 y[4,1]  y[4,2]  y[4,3]  y[4,4]  y[4,5]
 y[5,1]  y[5,2]  y[5,3]  y[5,4]  y[5,5]

In [8]:
@constraint(m, [i=1:I.n], sum(x[i,j] for j=1:I.n)==1)
@constraint(m, [j=1:I.n], sum(x[i,j] for i=1:I.n)==1)

@constraint(m, sum(x[i,j] for i=1:I.n, j=1:I.n)==I.n)

@constraint(m, [i=1:I.n], y[1,i] = sum(x[i,j]*j for j=1:I.n)) # for lookahed=1

for l=2:I.l
    @constraint(m, [i=1:I.n], y[l,i] = y[l-1,y[l-1,i]])
end

@objective(m, Min, sum(I.D[i,y[i,s]] for s=1:I.l, i=1:I.n))

# restrição remoção de subciclos

# @objective(m, Min, sum(I.D[pi[i],pi[i+s]]*x[pi[i],pi[i+s]] for s=1:I.l, i=1:I.n))

# @objective(m, Min, sum(D[i,j]*x[i,j] for i=1:n, j=1:n))
# @constraint(m, Min, sum(D[i,j]*x[i,j] for i=1:n, j=1:n))

LoadError: LoadError: At In[8]:6: `@constraint(m, [i = 1:I.n], y[1, i] = sum((x[i, j] * j for j = 1:I.n)))`: No constraint expression was given.
in expression starting at In[8]:6

In [11]:
optimize!(m)
objective_value(m)

2.3346591570662225

In [13]:
for i=1:n print(value(x[i]), " ") end

1.0 0.0 1.0 1.0 